In [13]:
import pprint
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import spotipy
import spotipy.util as util
from functools import reduce
from collections import Counter

## SETUP
# C:\MongoDB\bin\mongod.exe --dbpath=C:\MongoDB\data\db --directoryperdb
pp = pprint.PrettyPrinter()
client = MongoClient('localhost', 27017)
db = client.server

In [10]:
## Authentication
username = 'Ariel Godinho'
scope = 'user-top-read user-library-read playlist-modify-private playlist-read-private user-read-playback-state user-modify-playback-state'
token = util.prompt_for_user_token(
    username,
    scope,
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7',
    redirect_uri='http://localhost:8888/tree')

if token:
    #Set token to spotify library
    sp = spotipy.Spotify(auth=token)
    print("Token get successful. token", token)
else:
    print("Can't get token for", username)

Token get successful. token BQALod-bew_gQdO1ZwNQXb0c9laPYh1n-4AaWhjKj4Dn3LVIOFucJ4Ot4EyU5KCOFqBzUpcSiCKIxpx4RMY2oFFOwfi_79KTbazV8qNPDT9dgp30jg1vEKKfCk80lNUkFH2jlIFASWOMGa02QdFALJ1f70PaDofQLfpcU7n0rNqDi3gtRD7S0PyvKp57sm_qfWLxKJcdt-06ay-m4m3B


In [6]:
#Buscar artistas mais escutados
f_user_top_artists = sp.current_user_top_artists(50, 0, 'long_term')
pp.pprint([artist['name'] for artist in f_user_top_artists['items']])

# print(len(f_user_top_artists['items']))
#primeiro artista
# pp.pprint(f_user_top_artists['items'][0])

['Natiruts',
 'The Weeknd',
 'Daft Punk',
 'Years & Years',
 'Jorge & Mateus',
 'Drake',
 'Darren Korb',
 'Cacife Clandestino',
 'Muse',
 'Maneva',
 'Djavan',
 'Cidade Negra',
 'Disclosure',
 'Avicii',
 'Eminem',
 'Kamelot',
 'Calvin Harris',
 'Criolo',
 'Seu Jorge',
 'Os Paralamas Do Sucesso',
 'Red Hot Chili Peppers',
 'Lulu Santos',
 'Galantis',
 'Angra',
 'Major Lazer',
 'Black Sabbath',
 'Cássia Eller',
 'Bob Marley & The Wailers',
 'Projota',
 'David Guetta',
 'Incubus',
 'Symphony X',
 'Gorillaz',
 'Tiësto',
 'Michael Jackson',
 'Arctic Monkeys',
 'Ed Sheeran',
 'Queen',
 'Vicetone',
 'Maroon 5',
 'Nirvana',
 'Emicida',
 'Audioslave',
 'Zedd',
 'Justin Timberlake',
 'Tove Lo',
 'Charlie Brown Jr.',
 'Armin van Buuren',
 'Pink Floyd',
 'Michael Bublé']


In [7]:
#buscar seeds de generos possiveis
genre_seeds = sp.recommendation_genre_seeds()['genres']
genre_seeds.append('bossa')
genre_seeds.append('brazilian')
pp.pprint(genre_seeds)


['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [14]:
#Listar generos desses artistas

#Quebrar em palavras
top_genres = [artist['genres'] for artist in f_user_top_artists['items']]
# pp.pprint(top_genres)
top_genres_2d = reduce(lambda x, y: x+y, top_genres)
# pp.pprint(top_genres_2d)
top_genres_2d_split = [genre.split() for genre in top_genres_2d]
# pp.pprint(top_genres_2d_split)
top_genres_2d_one_word = reduce(lambda x, y: x+y, top_genres_2d_split)
# pp.pprint(top_genres_2d_one_word)


# todos os generos (para avaliar perda de generos)
# counter = Counter(top_genres_2d_one_word)
# common_genre_list = [genre[0] for genre in counter.most_common(5)]
# pp.pprint(counter)

# corrigindo generos relacionados
top_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in top_genres_2d_one_word]
top_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in top_genres_2d_one_word]


#Filtrar por generos que podem ser seed
filtered_top_genres = [x for x in top_genres_2d_one_word if x in genre_seeds]

#Contar generos que mais repetem
counter = Counter(filtered_top_genres)
pp.pprint(counter)

Counter({'rock': 46,
         'pop': 37,
         'metal': 25,
         'brazil': 19,
         'house': 19,
         'dance': 9,
         'edm': 9,
         'alternative': 8,
         'electro': 7,
         'mpb': 6,
         'reggae': 5,
         'funk': 4,
         'latin': 4,
         'samba': 3,
         'classical': 3,
         'trance': 3,
         'swedish': 3,
         'sertanejo': 2,
         'bossanova': 2,
         'pagode': 2,
         'indie': 2,
         'electronic': 1,
         'piano': 1,
         'afrobeat': 1,
         'garage': 1,
         'grunge': 1,
         'german': 1,
         'techno': 1})


In [15]:
#Pegar top 5 generos e buscar 50 ou 100 recomendacoes
common_genre_list = [genre[0] for genre in counter.most_common(5)]
pp.pprint(common_genre_list)

# Get spotify recommendations from genre seeds
recomendation_list = sp.recommendations(seed_genres=common_genre_list, limit=100)
pp.pprint([track['artists'][0]['name'] + ' - ' + track['name'] for track in recomendation_list['tracks']])
# print(recomendation_list['tracks'][0]['artists'][0])                                                            
    

['rock', 'pop', 'metal', 'brazil', 'house']
['Lulu Santos - Toda Forma de Amor',
 'Marshmello - Happier',
 "blink-182 - What's My Age Again?",
 'Kasabian - Ill Ray (The King)',
 'Ana Carolina - Quem de nós Dois (La Mia Storia Tra Le Dita)',
 'Avicii - Addicted To You',
 'Cazuza - Exagerado',
 'Sublime - What I Got',
 'Criolo - Não Existe Amor Em SP',
 'Lewis Del Mar - Painting (Masterpiece)',
 "The Maine - Don't Come Down",
 'Daft Punk - Derezzed',
 'Calvin Harris - Summer',
 'The Doors - L.A. Woman',
 'Of Mice & Men - Unbreakable',
 'Adelitas Way - Notorious',
 'Kehlani - Gangsta',
 'Camila Cabello - OMG',
 'Nightwish - Amaranth',
 'Stone Sour - Song #3',
 'The 1975 - Girls',
 'David Guetta - Bad (feat. Vassy) - Radio Edit',
 'Beastie Boys - Fight For Your Right',
 'Sabaton - To Hell and Back',
 'The Chainsmokers - Closer',
 'Calvin Harris - Thinking About You (feat. Ayah Marar)',
 'Creedence Clearwater Revival - Down On The Corner',
 'Los Hermanos - Último Romance - Ao Vivo',
 'Blast

In [17]:
#Avaliacao

#avalia se as musicas recomendadas estao no bd (+1 no rating) e se o artista esta no bd (+1 rating)
dict_music_id_rating = {}
artist_list = []
rating = 0
for i in range(len(recomendation_list['tracks'])):
    rating = 0
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']
    
    #musica na base
    track = db.tracks.find_one({'id' : track_id})
    if (track is not None):
        rating = rating + 1
#     print(track)
    
    #artista na base
    artist = db.tracks.find_one({'artists_ids' : artist_id})
    if (artist is not None):
        rating = rating + 1
#     print(artist)
    
    artist_list.append(artist_id)
    dict_music_id_rating[track_id] = rating

#dicionario 'musica id': 'rating'
print(dict_music_id_rating)


{'2ma8V5vEOfCuyqZ0V8HAq3': 2, '2dpaYNEQHiRxtZbfNsse99': 2, '5wTMQjEx9LWeFwLjolPCUy': 1, '0WruOD5fP4RJ1kAn3Pcv3h': 2, '2yQMmtem48vOjB1OXAvbKD': 2, '0xIVvRmjztR1AwuHrkhH41': 2, '4d0DpU7Odiv0ztvX2GxJlk': 2, '5nkYHBWXu7KCgcdwP6jcB1': 1, '7mrL741YwpaiRKg56zakhb': 1, '4kK14radw0XfwxJDPt9tnP': 2, '47ojH5LeQhl1ZltSAHBEFF': 2, '4bJhvPYFBlyoUOo8ts3b6B': 1, '7b71WsDLb8gG0cSyDTFAEW': 1, '6DmfWj5kOa1fX8AwN9byOn': 2, '15TC83LO1QAu0BZLRdHw01': 0, '5PZo9NMyNLxOlpEFoYo611': 0, '1W7zkKgRv9mrLbfdQ8XyH3': 2, '4xBjsLUSjcx2h7PcRikpQp': 1, '0cYduKz7njWaJ73PGn6OGW': 1, '5jsDxDkJ1PqyYUWhDMr86B': 2, '5WSdMcWTKRdN1QYVJHJWxz': 0, '6PtXobrqImYfnpIxNsJApa': 1, '5NLuC70kZQv8q34QyQa1DP': 2, '3RVg6RZvfOLrw4pqjVNbQr': 0, '7BKLCZ1jbUBVqRi2FVlTVw': 2, '1KtD0xaLAikgIt5tPbteZQ': 1, '2gE95JskwQ1pCACTpGe1Db': 2, '3Pssl5CAjQVYMJDlPphGUb': 1, '0kk91i9mALCx4joZGk4nHT': 1, '0NSqYlz5V1u6l68r1pzUPn': 0, '2CWnuF7ht55Ajyeg4WPvWu': 1, '5HAVhn8razv1yHLgry3CAT': 1, '4F7A0DXBrmUAkp32uenhZt': 2, '1e1JKLEDKP7hEQzJfNAgPl': 2, '6WakVarAzRjM

In [19]:
#busca os dados dos artista no spotify 

#Para ser mais eficiente, poderia ser buscado no bd e se nao existisse buscado no spotify
#teria que criar um documento no bd do tipo artista e não apenas referencia-lo no documento track

artists_info_1 = sp.artists(artist_list[0:50])
artists_info_2 = sp.artists(artist_list[50:100])
artists_info = artists_info_1['artists'] + artists_info_2['artists']

In [20]:
#Finalizando a avaliacao

#Cada genero do artista no tops generos do usuario (+1 no rating)

dict_ratings = {}
artist_id_genre_count = {} # dict de id_artista e quantidade de top generos do usuario
for i in range(len(artists_info)):
    
    artist_id = artists_info[i]['id']
    artist_genres = artists_info[i]['genres']
#     print(artist_genres)
    
    if len(artist_genres) != 0:
        artist_genres_2d_split = [genre.split() for genre in artist_genres]
#         print(artist_genres_2d_split)
        artist_genres_2d_one_word = reduce(lambda x, y: x+y, artist_genres_2d_split)
#         print(artist_genres_2d_one_word)
        artist_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in artist_genres_2d_one_word]
        artist_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in artist_genres_2d_one_word]
#         print(artist_genres_2d_one_word)
    
    filtered_by_user_top_genres = [x for x in artist_genres_2d_one_word if x in common_genre_list]
    filtered_by_user_top_genres = list(set(filtered_by_user_top_genres))
#     print(filtered_by_user_top_genres)
#     print(len(filtered_by_user_top_genres))
    
    artist_id_genre_count[artist_id] = len(filtered_by_user_top_genres)

# verificar se o dict de idArtista + qtd Genero e dict idMusica + rating estao ok
# print(artist_id_genre_count)    
# print(dict_music_id_rating)

for i in range(len(recomendation_list['tracks'])):
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']

    #somando ratings para ter o rating total
    dict_ratings[track_id] = dict_music_id_rating[track_id] + artist_id_genre_count[artist_id]
    
#ordernar por maior pontuacao
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
ordenado = sorted(dict_ratings.items(), key=operator.itemgetter(1), reverse=True)

#top 10 musicas recomendadas
print(ordenado[0:10])
    
    

[('0uybt73QFXaLCoxuVf6fhm', 5), ('2ma8V5vEOfCuyqZ0V8HAq3', 4), ('2dpaYNEQHiRxtZbfNsse99', 4), ('4d0DpU7Odiv0ztvX2GxJlk', 4), ('47ojH5LeQhl1ZltSAHBEFF', 4), ('5jsDxDkJ1PqyYUWhDMr86B', 4), ('7BKLCZ1jbUBVqRi2FVlTVw', 4), ('6p2liQLGoDaLXgND68bfVt', 4), ('1oc3jna2ablnfIKLYbKujl', 4), ('3u1S1OmAUhx5DRlLrXqyp3', 4)]
